In [38]:
import torch
import albumentations
import os
import cv2
import csv
import timm
import pickle
import numpy as np
import pandas as pd
from zipfile import ZipFile
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split
from glob import glob
from zipfile import ZipFile
from PIL import Image
from albumentations.pytorch.transforms import ToTensorV2
from albumentations import augmentations as A
from sklearn.preprocessing import LabelEncoder
from torchvision import models, transforms
from torchmetrics.functional import accuracy

In [39]:
base_path = "D:\Art DataBase"

In [40]:
train_path = os.path.join(base_path, 'train', '*')

In [41]:
test_path = os.path.join(base_path, 'test', '*')

In [42]:
test_data = glob(test_path)

In [43]:
train_data = glob(train_path)

In [44]:
print(type(train_data))

<class 'list'>


In [45]:
metadata = pd.read_csv(os.path.join(base_path, 'metadata.csv'))

In [46]:
norm_mean = (0.485, 0.456, 0.406)
norm_std = (0.229, 0.224, 0.225)

transforms = albumentations.Compose(
    [A.Normalize(norm_mean, norm_std),
     A.geometric.Resize(256, 256),
     ToTensorV2()]
)

In [47]:
#The fit method(standardization) is calculating the mean and variance of 
#each of the features
#present in our data. The transform
#method is transforming all the features using the respective mean and variance.
#labelencoder = Encode target labels with value between 0 and n_classes-1.
enc = LabelEncoder()

metadata['artist_cat'] = enc.fit_transform(metadata['artist'].astype(str))
metadata['style_cat'] = enc.fit_transform(metadata['style'].astype(str))
metadata['filename'] = enc.fit_transform(metadata['new_filename'].astype(str))

In [48]:
metadata[metadata.new_filename == train_data[0].split('\\')[-1]]['artist_cat'].to_numpy()[0]
metadata

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename,artist_cat,style_cat,filename
0,Barnett Newman,1955.0,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg,270,18,2512
1,Barnett Newman,1950.0,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg,270,18,75733
2,kiri nichol,2013.0,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg,2255,81,27859
3,kiri nichol,2013.0,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg,2255,81,14703
4,kiri nichol,2013.0,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg,2255,81,85029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103245,Jackson Pollock,1948.0,abstract,682.0,220.0,96405.0,wikiart,Action painting,Number 13A (Arabesque),train_and_test,True,25525.jpg,1000,3,20503
103246,Bernardo Strozzi,NaN,religious painting,329.0,456.0,127594.0,wikiart,Baroque,St. Francis of Assisi,train_only,True,47038.jpg,296,12,44406
103247,Josef Sima,NaN,landscape,293.0,512.0,102519.0,wikiart,Surrealism,Maisons à la campagne II,train_and_test,False,7680.jpg,1163,120,77474
103248,Brett Whiteley,1982.0,marina,293.0,512.0,167423.0,wikiart,NaN,Thebe's Revenge,train_and_test,True,9021.jpg,329,136,92375


In [49]:
class ArtistDataset(Dataset):
    
    def __init__(self, base_path, transforms=None):
        self.base_path = base_path
        self.data = glob(self.base_path)
        self.metadata = pd.read_csv('D:\Art DataBase\metadata.csv')
        self.transforms = transforms
    
    def __getitem__(self, idx):
        img_path = self.data[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transforms is not None:
            img = self.transforms(image=img)['image']
        return img, torch.tensor(metadata[metadata.new_filename == img_path.split('\\')[-1]]['artist_cat'].to_numpy()[0])
    
    def __len__(self):
        return len(self.data)
        

In [50]:
train_dataset = ArtistDataset(train_path, transforms=transforms)

In [51]:
test_dataset = ArtistDataset(test_path, transforms=transforms)

In [52]:
train_size = int(0.8 * len(train_dataset))

In [53]:
val_size = len(train_dataset) - train_size

In [54]:
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])


In [55]:
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=32,
                          shuffle=True)

In [56]:
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=32,
                          shuffle=True)

In [57]:
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=32,
                          shuffle=True)

In [58]:
model = timm.create_model(model_name='resnet50', pretrained=True)

In [59]:
last_fc = nn.Linear(in_features=2048, out_features=2319)

In [60]:
model.fc = last_fc

In [61]:
params = model.parameters()


In [62]:
#optimizer = optim.SGD(params, lr = 1e-2)
optimizer = torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [63]:
loss = nn.CrossEntropyLoss()

In [64]:
#model(train_dataset[0][0].unsqueeze(dim=0))

In [65]:
#for i,data in enumerate(train_loader,0):    
#    x, y = data
#    logit = model(x)

In [35]:
#training and validation loops
n_epochs = 1
for epoch in range(n_epochs):
    losses = list()
    accuracies = list()
    model.train() #because of dropout in train we want it
    for i,data in enumerate(train_loader,0):
        
        
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()            
        
        labels = labels.long()
        
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        outputs = model(inputs)
        
        J = loss(outputs, labels)
        
        #import pdb; pdb.set_trace() ##for debugging
        
        J.backward()
        optimizer.step()
      
        losses.append(J.item())
        accuracies.append(labels.eq(outputs.detach().argmax(dim=1).cpu()).float().mean())
    
        print(f'Epoch {epoch + 1}', end=', ')
        print(f'train loss: {torch.tensor(losses).mean():.2f}', end=', ')
        print(f'train acc: {torch.tensor(accuracies).mean():.9f}')
    
    losses = list()
    model.eval()#because of dropout in eval we dont
    for i,data in enumerate(val_loader,0):
        
        inputs, labels = data
        labels = labels.long()
        inputs, labels = inputs.cuda(), labels.cuda()            
        

        
        with torch.no_grad():
            outputs = model(inputs)
        
        #2 compute objecttive function (howw well the network does)
        J = loss(outputs, labels)
        
        losses.append(J.item())
        accuracies.append(labels.eq(outputs.detach().argmax(dim=1).cpu()).float().mean())
    
        print(f'Epoch {epoch + 1}', end=', ')
        print(f'val loss: {torch.tensor(losses).mean():.2f}', end=', ')
        print(f'val acc: {torch.tensor(accuracies).mean():.9f}')

    

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [ ]:
#torch.save(model.state_dict(), os.path.join('D:\Art DataBase\models','tryandeval.pth'))

In [66]:
net = model
model.load_state_dict(torch.load(r'D:\Art DataBase\models\tryandeval.pth'))


<All keys matched successfully>

In [67]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 36 %
